In [7]:
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 100.0 MB/s eta 0:00:00


In [3]:
import ipywidgets as widgets
from IPython.display import display

def create_config_dashboard(config=None):
    # Model architecture parameters
    d_model = widgets.IntSlider(min=64, max=1024, step=64, value=512, description='d_model:')
    num_heads = widgets.Dropdown(
        options=[1, 2, 4, 8, 16], 
        value=8, 
        description='Heads:'
    )
    
    # Automatically update valid num_heads options when d_model changes
    def update_heads_options(change):
        valid_heads = [h for h in [1, 2, 4, 8, 16] if change['new'] % h == 0]
        num_heads.options = valid_heads
        
    d_model.observe(update_heads_options, names='value')
    
    # Training parameters
    lr = widgets.FloatLogSlider(min=-5, max=-2, value=-3.5, description='LR:')
    warmup = widgets.IntSlider(min=0, max=10000, step=100, value=4000, description='Warmup:')
    
    # Validation indicator
    validation = widgets.Valid(value=True, description='Valid config')
    
    # Output area for validation messages
    output = widgets.Output()
    
    # Layout all components
    dashboard = widgets.VBox([
        widgets.HBox([d_model, num_heads]),
        widgets.HBox([lr, warmup]),
        validation,
        output
    ])
    
    return dashboard

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def create_lr_schedule_dashboard():
    # Interactive parameters
    scheduler_type = widgets.Dropdown(
        options=['inverse_sqrt', 'cosine', 'linear', 'constant'],
        value='inverse_sqrt',
        description='Scheduler:'
    )
    
    base_lr = widgets.FloatLogSlider(
        min=-5, max=-2, value=-3.5, 
        description='Base LR:'
    )
    
    warmup_steps = widgets.IntSlider(
        min=100, max=10000, step=100, value=4000,
        description='Warmup:'
    )
    
    total_steps = widgets.IntSlider(
        min=5000, max=100000, step=1000, value=50000,
        description='Total steps:'
    )
    
    # Output plot
    plot_output = widgets.Output()
    
    # Update function
    def update_plot(change=None):
        plot_output.clear_output(wait=True)
        with plot_output:
            fig, ax = plt.subplots(figsize=(10, 6))
            steps = np.arange(1, total_steps.value + 1)
            
            # Calculate learning rate for each step based on scheduler type
            if scheduler_type.value == 'inverse_sqrt':
                lr_values = [min(step**(-0.5), step*(warmup_steps.value**(-1.5))) 
                           * warmup_steps.value**0.5 * 10**base_lr.value for step in steps]
            elif scheduler_type.value == 'cosine':
                lr_values = []
                for step in steps:
                    if step < warmup_steps.value:
                        # Linear warmup
                        lr = (step / warmup_steps.value) * 10**base_lr.value
                    else:
                        # Cosine annealing
                        progress = (step - warmup_steps.value) / (total_steps.value - warmup_steps.value)
                        lr = 0.5 * (1 + np.cos(np.pi * progress)) * 10**base_lr.value
                    lr_values.append(lr)
            
            ax.plot(steps, lr_values)
            ax.set_xlabel('Training steps')
            ax.set_ylabel('Learning rate')
            ax.set_title(f'Learning Rate Schedule: {scheduler_type.value}')
            ax.grid(True)
            plt.tight_layout()
            plt.show()
    
    # Connect controls to update function
    scheduler_type.observe(update_plot, names='value')
    base_lr.observe(update_plot, names='value')
    warmup_steps.observe(update_plot, names='value')
    total_steps.observe(update_plot, names='value')
    
    # Initial plot
    update_plot()
    
    # Layout
    controls = widgets.VBox([scheduler_type, base_lr, warmup_steps, total_steps])
    dashboard = widgets.HBox([controls, plot_output])
    
    return dashboard

In [8]:
import plotly.express as px
import pandas as pd
import ipywidgets as widgets

def visualize_model_parameters(model):
    # Extract model parameters by component
    param_count = {}
    total_params = 0
    
    for name, module in model.named_modules():
        if len(list(module.children())) == 0:  # Leaf module
            num_params = sum(p.numel() for p in module.parameters() if p.requires_grad)
            if num_params > 0:
                param_count[name] = num_params
                total_params += num_params
    
    # Create hierarchical representation
    param_data = []
    for name, count in param_count.items():
        parts = name.split('.')
        # Create path for treemap
        for i in range(1, len(parts) + 1):
            path = '.'.join(parts[:i])
            size = count if i == len(parts) else 0
            param_data.append({'path': path, 'size': size})
    
    # Aggregate data for visualization
    df = pd.DataFrame(param_data)
    
    # Create treemap
    fig = px.treemap(
        df, 
        path=[col for col in df['path'].str.split('.', expand=True).columns], 
        values='size',
        title=f'Model Parameter Distribution (Total: {total_params:,} parameters)',
        color_continuous_scale='Viridis'
    )
    
    fig.update_layout(
        margin=dict(l=0, r=0, t=30, b=0),
        height=600,
    )
    
    return fig


In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

def create_experiment_comparison(experiment_results):
    # Create experiment comparison dashboard
    # Assuming experiment_results is a list of dicts with metrics and configs
    
    # Extract experiment data
    exp_names = [exp['name'] for exp in experiment_results]
    
    # Create performance comparison subplot
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("Training Loss", "Validation Loss", "Perplexity", "Training Time")
    )
    
    # Add traces for each metric
    for i, exp in enumerate(experiment_results):
        # Training loss
        fig.add_trace(
            go.Scatter(x=list(range(len(exp['train_loss']))), 
                      y=exp['train_loss'],
                      mode='lines',
                      name=f"{exp['name']} - Train Loss"),
            row=1, col=1
        )
        
        # Validation loss
        fig.add_trace(
            go.Scatter(x=list(range(len(exp['val_loss']))), 
                      y=exp['val_loss'],
                      mode='lines',
                      name=f"{exp['name']} - Val Loss"),
            row=1, col=2
        )
        
        # Perplexity
        fig.add_trace(
            go.Scatter(x=list(range(len(exp['perplexity']))), 
                      y=exp['perplexity'],
                      mode='lines',
                      name=f"{exp['name']} - Perplexity"),
            row=2, col=1
        )
    
    # Add training time comparison
    times = [exp['training_time'] for exp in experiment_results]
    fig.add_trace(
        go.Bar(x=exp_names, y=times, name="Training Time (hours)"),
        row=2, col=2
    )
    
    # Update layout
    fig.update_layout(height=700, width=1000, title_text="Experiment Comparison")
    
    # Create parameter comparison table
    param_data = []
    for exp in experiment_results:
        params = {
            'Name': exp['name'],
            'd_model': exp['config']['d_model'],
            'num_heads': exp['config']['num_heads'],
            'num_layers': exp['config']['num_layers'],
            'batch_size': exp['config']['batch_size'],
            'learning_rate': exp['config']['learning_rate'],
            'best_val_loss': min(exp['val_loss']),
            'best_perplexity': min(exp['perplexity']),
        }
        param_data.append(params)
    
    param_df = pd.DataFrame(param_data)
    
    return fig, param_df

In [10]:
import ipywidgets as widgets
from IPython.display import display

class TrainingInsightDashboard:
    def __init__(self, model=None, config=None):
        self.model = model
        self.config = config or {}
        self.experiment_history = []
        
        # Create tabs for different dashboards
        self.tab = widgets.Tab()
        
        # Initialize all dashboards
        self.config_dashboard = self.create_config_dashboard()
        self.arch_dashboard = self.create_architecture_dashboard()
        self.training_dashboard = self.create_training_dashboard()
        self.resource_dashboard = self.create_resource_dashboard()
        self.comparison_dashboard = self.create_comparison_dashboard()
        
        # Set up tabs
        self.tab.children = [
            self.config_dashboard,
            self.arch_dashboard,
            self.training_dashboard,
            self.resource_dashboard,
            self.comparison_dashboard
        ]
        
        # Set tab titles
        titles = ['Configuration', 'Architecture', 'Training', 
                 'Resources', 'Comparison']
        for i, title in enumerate(titles):
            self.tab.set_title(i, title)
    
    def create_config_dashboard(self):
        # Implementation as shown above
        return widgets.VBox([widgets.Label("Configuration Dashboard")])
    
    def create_architecture_dashboard(self):
        # Implementation as shown above
        return widgets.VBox([widgets.Label("Architecture Dashboard")])
    
    def create_training_dashboard(self):
        # Implementation as shown above
        return widgets.VBox([widgets.Label("Training Dashboard")])
    
    def create_resource_dashboard(self):
        # Implementation as shown above
        return widgets.VBox([widgets.Label("Resource Dashboard")])
    
    def create_comparison_dashboard(self):
        # Implementation as shown above
        return widgets.VBox([widgets.Label("Comparison Dashboard")])
    
    def display(self):
        return self.tab

In [18]:
# Import necessary libraries
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from IPython.display import display
import ipywidgets as widgets

# Import your project components
from src.models.transformer import EncoderDecoderTransformer
from src.training.transformer_trainer import TransformerTrainer

# Import the dashboard class (assuming it's in src/utils/dashboard.py)
#from src.utils.dashboard import TrainingInsightDashboard

# CELL 1: Initialize the dashboard
# ---------------------------------
# Create a new dashboard instance
dashboard = TrainingInsightDashboard()

# Display the dashboard
display(dashboard.display())

In [19]:
# CELL 2: Configure a model using the dashboard
# ---------------------------------------------
# Get configuration from dashboard
config = dashboard.get_current_config()
print("Current configuration:")
for k, v in config.items():
    print(f"  {k}: {v}")

# Create a model with this configuration
model = EncoderDecoderTransformer(
    src_vocab_size=config['src_vocab_size'],
    tgt_vocab_size=config['tgt_vocab_size'],
    d_model=config['d_model'],
    num_heads=config['num_heads'],
    num_encoder_layers=config['num_encoder_layers'],
    num_decoder_layers=config['num_decoder_layers'],
    d_ff=config['d_ff'],
    dropout=config['dropout']
)

# Update the dashboard with the created model
dashboard.update_model(model)

# Now the Architecture tab will show visualizations of the model

AttributeError: 'TrainingInsightDashboard' object has no attribute 'get_current_config'